<a href="https://colab.research.google.com/github/smdbg/colab/blob/main/lance_API_Server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import lancedb
import uvicorn

# --------------------------------------------------------------
# 1. Дефиниция на приложението
# --------------------------------------------------------------
app = FastAPI(title="LanceDB Remote Server")

# Път към данните на сървъра
DB_URI = "data/lancedb"
TABLE_NAME = "docling"

# Модел на заявката
class SearchQuery(BaseModel):
    text: str
    limit: int = 5

# --------------------------------------------------------------
# 2. Ендпойнт за търсене
# --------------------------------------------------------------
@app.post("/search")
def search_db(query: SearchQuery):
    try:
        db = lancedb.connect(DB_URI)

        if TABLE_NAME not in db.table_names():
            raise HTTPException(status_code=404, detail="Table not found")

        table = db.open_table(TABLE_NAME)

        # Изпълнение на търсенето
        result = table.search(query.text, query_type="vector").limit(query.limit)
        df = result.to_pandas()

        # Връщаме резултата като JSON
        # Премахваме векторната колона, защото е твърде голяма за мрежов трафик
        if 'vector' in df.columns:
            df = df.drop(columns=['vector'])

        return df.to_dict(orient="records")

    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

# --------------------------------------------------------------
# 3. Стартиране на сървъра
# --------------------------------------------------------------
if __name__ == "__main__":
    # Това ще пусне сървъра на порт 8000 и ще е достъпен от мрежата (0.0.0.0)
    print("🚀 LanceDB Server running on port 8000...")
    uvicorn.run(app, host="0.0.0.0", port=8000)